In [7]:
import json
import re
import requests
import shutil

url = "https://raw.githubusercontent.com/smogon/pokemon-showdown/master/data/pokedex.ts"
response = requests.get(url)
# response = json.loads(response.text)
with open("dex.ts", "w") as file:
    file.write(response.text)

In [20]:
import re

txt = ""
with open("dex.ts") as file:
    txt = file.read()

txt = re.sub("//.*\n", "\n", txt)
txt = re.split("=", txt)[1]
txt = re.sub("(\s|{)(\w+):", "\g<1>\"\g<2>\":", txt)
txt = re.sub(",(\s*)(]|})", "\g<1>\g<2>", txt)
txt = re.sub(";", "", txt)

with open("dex.json", "w") as file:
    file.write(txt)


In [22]:
import pandas as pd

def get_gen(num):
    gen_nums = [(0, 0), (1, 151), (2, 251), (3, 386), (4, 493), (5, 649), (6, 721), (7, 809), (8, 905), (9, 1008)]
    for gen, num_max in gen_nums:
        if num <= num_max:
            return gen

df = pd.read_json("dex.json", orient="index")
df = df.join(pd.DataFrame(df.types.tolist(), index=df.index).stack().str.get_dummies().groupby(level=0).sum())
df = df.join(pd.DataFrame(df.eggGroups.tolist(), index=df.index).stack().str.get_dummies().groupby(level=0).sum().add_prefix("eggGroup_"))
df = df.join(pd.json_normalize(df.genderRatio).set_index(df.index))
df = df.join(pd.json_normalize(df.baseStats).set_index(df.index))
df = df.join(pd.json_normalize(df.abilities).set_index(df.index).add_prefix("ability_"))
df = df.join(df["tags"].dropna().apply(lambda x: x[0]).rename("legendary"))
df = df.join(df["evos"].dropna().apply(len).rename("numEvos"))
df.loc[df.gender == "M", ["M", "F"]] = [1, 0]
df.loc[df.gender == "F", ["M", "F"]] = [0, 1]
df.loc[df.gender == "N", ["M", "F"]] = [0, 0]
df["gen"] = df["num"].apply(get_gen)
df.loc[df.forme == "Mega", "gen"] = 6
df.loc[df.forme == "Primal", "gen"] = 6
df.loc[df.forme == "Alola", "gen"] = 7
df.loc[df.forme == "Alola-Totem", "gen"] = 7
df.loc[df.forme == "Galar", "gen"] = 8
df.loc[df.forme == "Gmax", "gen"] = 8
df.loc[df.forme == "Hisui", "gen"] = 8
df.drop(["types", "genderRatio", "baseStats", "abilities", "eggGroups", "gender", "tags", "Bird", "evos"], axis=1, inplace=True)
df.to_csv("dex.csv", index=False)
df

,num,name,heightm,weightkg,color,prevo,evoLevel,otherFormes,formeOrder,canGigantamax,...,def,spa,spd,spe,ability_0,ability_H,ability_1,ability_S,legendary,numEvos
bulbasaur,1,Bulbasaur,0.7,6.9,Green,NaN,NaN,NaN,NaN,NaN,...,49,65,65,45,Overgrow,Chlorophyll,NaN,NaN,NaN,1.0
ivysaur,2,Ivysaur,1.0,13.0,Green,Bulbasaur,16.0,NaN,NaN,NaN,...,63,80,80,60,Overgrow,Chlorophyll,NaN,NaN,NaN,1.0
venusaur,3,Venusaur,2.0,100.0,Green,Ivysaur,32.0,[Venusaur-Mega],"[Venusaur, Venusaur-Mega]",G-Max Vine Lash,...,83,100,100,80,Overgrow,Chlorophyll,NaN,NaN,NaN,NaN
venusaurmega,3,Venusaur-Mega,2.4,155.5,Green,NaN,NaN,NaN,NaN,NaN,...,123,122,120,80,Thick Fat,NaN,NaN,NaN,NaN,NaN
venusaurgmax,3,Venusaur-Gmax,2.0,0.0,Green,NaN,NaN,NaN,NaN,NaN,...,83,100,100,80,Overgrow,Chlorophyll,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pokestarspirit,-5011,Pokestar Spirit,1.5,61.0,White,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,Wonder Guard,NaN,NaN,NaN,NaN,NaN
pokestarblackdoor,-5012,Pokestar Black Door,1.5,61.0,White,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,Early Bird,NaN,NaN,NaN,NaN,NaN
pokestarwhitedoor,-5013,Pokestar White Door,1.5,61.0,White,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,Blaze,NaN,NaN,NaN,NaN,NaN
pokestarblackbelt,-5014,Pokestar Black Belt,1.5,61.0,White,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,Huge Power,NaN,NaN,NaN,NaN,NaN
